In [3]:
import csv
import json
import os 
import uuid
from urllib import parse 

##Open directory and create list of files to read
dir_to_read = 'user_research/CSV'
files_to_read = os.listdir(dir_to_read)

##Write json file
jsonfile = open('user_research/JSON/DN_INFLUENCER.json', 'w', encoding='utf-8')

author_map_filename = 'author_map.json'

if os.path.exists(author_map_filename):
    print('Using existing author map')
    with open(author_map_filename, 'r') as f:
        author_map = json.load(f)
else:
    print('Creating new author map. Results will not be joinable with previous data!')
    author_map = dict()


def video_id(url):
    """ Convert a Youtube URL to a video id
    """
    querystring = parse.urlparse(url).query
    qs_dict = parse.parse_qs(querystring)
    return qs_dict.get('v')[0]


## There are some hidden files to sync Dropbox, avoid them
for filename in files_to_read:
    if filename[-4:] != '.csv':
        continue
        
    csvfile = open(f'{dir_to_read}/{filename}', 'r', encoding='utf-8')
    
    reader = csv.DictReader(csvfile, restkey = 'to_delete', delimiter = ';') #, fieldnames)

##Dump reader object to json string, looping through
    try:
        for i, row in enumerate(reader):
            author_id = author_map.get(row['author'])
            if author_id is None:
                author_id = str(uuid.uuid4())
                author_map[row['author']] = author_id
            
            del row['id']
            row['author'] = author_id
            row['video_id'] = video_id(row['link'])
            if 'to_delete' in row.keys():
                del row['to_delete']
            row = dict(row)
            row['r2r'] = row.pop('to')
            json.dump(row, jsonfile)
            jsonfile.write('\n')
    except Exception as e:
        print(f'Failed to read {filename}, row {i}')
        print(e)
        
## Writing dict to json
with open(author_map_filename, 'w') as f:
    json.dump(author_map, f)

Using existing author map
